In [1]:
import pybaseball
from pybaseball import cache
from pybaseball import statcast
import pandas as pd

In [2]:
statcast_data = statcast('2025-03-28', '2025-03-29')

This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:06<00:00,  3.04s/it]


In [3]:
statcast_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
2788,FF,2025-03-29,93.8,-1.78,5.51,"Pagán, Emilio",682641,641941,field_out,hit_into_play,...,1,1.3,0.51,0.51,42.0,4.597651,-4.962009,23.996137,50.033604,37.403374
2904,FF,2025-03-29,95.3,-1.8,5.52,"Pagán, Emilio",527038,641941,field_out,hit_into_play,...,1,1.16,0.59,0.59,41.7,5.850287,3.003886,32.663956,38.065149,27.147809
3038,FF,2025-03-29,94.4,-1.78,5.59,"Pagán, Emilio",527038,641941,None,ball,...,1,1.16,0.62,0.62,42.2,<NA>,<NA>,<NA>,<NA>,<NA>
3170,FS,2025-03-29,84.0,-1.58,5.52,"Pagán, Emilio",527038,641941,None,ball,...,1,2.64,1.0,1.0,45.7,<NA>,<NA>,<NA>,<NA>,<NA>
3298,FF,2025-03-29,93.3,-1.71,5.61,"Pagán, Emilio",527038,641941,None,swinging_strike,...,1,1.12,0.44,0.44,39.5,4.422645,3.885418,23.835304,24.165759,24.090179


In [4]:
# cols i want:
cols=['game_date', 'pitch_type', 'p_throws', 'stand', 'events', 'description', 'p_throws', 'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'player_name', 'HB', 'IVB', 'plate_x', 'plate_z', 'effective_speed', 'release_spin_rate', 'release_extension', 'pitcher', 'spin_axis']

In [5]:
filt_data = statcast_data[cols]

KeyError: "['HB', 'IVB'] not in index"

In [ ]:
# calculating (approx) VAA and HAA

import numpy as np

def calculate_vaa_haa(df):
    # horizontal (x), vertical (z), and distance to plate (y)
    dx = df['plate_x'] - df['release_pos_x']
    dz = df['plate_z'] - df['release_pos_z']
    dy = -df['release_pos_y']  # plate at y = 0

    df['VAA'] = np.degrees(np.arctan2(dz, np.abs(dy)))
    df['HAA'] = np.degrees(np.arctan2(dx, np.abs(dy)))

    return df

filt_data = calculate_vaa_haa(filt_data)
filt_data.describe()


/var/folders/h6/_rz_hgwd06q2j4rsml2mr9yr0000gn/T/ipykernel_37285/3118122403.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VAA'] = np.degrees(np.arctan2(dz, np.abs(dy)))
/var/folders/h6/_rz_hgwd06q2j4rsml2mr9yr0000gn/T/ipykernel_37285/3118122403.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HAA'] = np.degrees(np.arctan2(dx, np.abs(dy)))


,game_date,release_speed,release_pos_x,release_pos_y,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,effective_speed,release_spin_rate,release_extension,pitcher,spin_axis,VAA,HAA
count,6916,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6883.0,6885.0,6885.0,6916.0,6885.0,6885.0,6885.0
mean,2025-03-28 15:16:33.175245824,89.538141,-1.045949,54.114065,5.728649,-0.150144,0.600076,0.050731,2.313275,89.561572,2287.153522,6.385882,644113.063331,176.157008,-3.610221,1.161932
min,2025-03-28 00:00:00,55.3,-4.27,52.34,3.64,-1.88,-1.74,-2.98,-1.46,52.8,418.0,3.8,434378.0,1.0,-8.068986,-6.727209
25%,2025-03-28 00:00:00,85.4,-2.26,53.83,5.48,-0.93,0.11,-0.51,1.68,85.3,2148.0,6.1,622491.0,128.0,-4.313438,0.319016
50%,2025-03-29 00:00:00,90.3,-1.73,54.11,5.78,-0.29,0.61,0.04,2.32,90.5,2331.0,6.4,663559.0,204.0,-3.595623,1.674714
75%,2025-03-29 00:00:00,94.2,-0.67,54.41,6.01,0.55,1.24,0.62,2.98,94.2,2491.0,6.7,678821.0,225.0,-2.882813,2.586682
max,2025-03-29 00:00:00,102.8,4.65,56.71,7.31,2.12,1.96,3.8,6.37,104.5,3435.0,8.2,808967.0,360.0,0.669362,6.633709
std,NaN,5.740167,1.801798,0.448786,0.463881,0.911672,0.716934,0.836298,0.96937,5.893679,370.693313,0.44979,61751.233032,72.091796,1.097972,2.090709


In [ ]:
filt_data.columns

Index(['game_date', 'pitch_type', 'p_throws', 'stand', 'events', 'description',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z',
       'player_name', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
       'effective_speed', 'release_spin_rate', 'release_extension', 'pitcher',
       'spin_axis', 'VAA', 'HAA'],
      dtype='object')

In [ ]:
# define events of interest: 
swinging_strikes = {
    'swinging_strike',
    'swinging_strike_blocked'
}

swings = {
    'foul', 'foul_tip', 'swinging_strike',
    'swinging_strike_blocked', 'hit_into_play'
}

called_strikes = {'called_strike'}
balls = {'ball', 'blocked_ball'}


In [ ]:
# Aggregating predictive stats per pitcher:
df=filt_data.copy()
df['season'] = df['game_date'].dt.year

# binary cols:
df['is_swing'] = df['description'].isin(swings)
df['is_whiff'] = df['description'].isin(swinging_strikes)
df['is_called_strike'] = df['description'].isin(called_strikes)
df['is_ball'] = df['description'].isin(balls)

# aggregate to pitcher-pitch_type-season level:
agg = (
    df.groupby(['pitcher', 'pitch_type', 'season'])
      .agg(
          pitches=('description', 'count'),
          swings=('is_swing', 'sum'),
          whiffs=('is_whiff', 'sum'),
          called_strikes=('is_called_strike', 'sum'),
          balls=('is_ball', 'sum')
      )
      .reset_index()
)

agg['whiff_pct'] = agg['whiffs'] / agg['swings']
agg['csw_pct'] = (agg['whiffs'] + agg['called_strikes']) / agg['pitches']
agg.head()



,pitcher,pitch_type,season,pitches,swings,whiffs,called_strikes,balls,whiff_pct,csw_pct
0,434378,CH,2025,9,2,1,1,6,0.500000,0.222222
1,434378,CU,2025,10,3,1,4,3,0.333333,0.500000
2,434378,FF,2025,43,22,4,7,14,0.181818,0.255814
3,434378,SL,2025,10,7,2,1,2,0.285714,0.300000
4,434378,ST,2025,11,4,0,3,4,0.000000,0.272727


In [ ]:
# aggregating descriptors to season level too
desc = (
    df.groupby(['pitcher', 'pitch_type', 'season'])
      .agg(
          n_pitches=('description', 'count'),
          velo=('release_speed', 'mean'),
          pfx_x=('pfx_x', 'mean'),
          pfx_z=('pfx_z', 'mean'),
          VAA=('VAA', 'mean'),
          HAA=('HAA', 'mean'),
          ext=('release_extension', 'mean'),
          spin=('release_spin_rate', 'mean'),
          rel_x=('release_pos_x', 'mean'),
          rel_z=('release_pos_z', 'mean'),
          plate_x=('plate_x', 'mean'),
          plate_z=('plate_z', 'mean'),
      )
      .reset_index()
)

# usage
total = desc.groupby(['pitcher', 'season'])['n_pitches'].transform('sum')
desc['usage'] = desc['n_pitches'] / total
desc.head()

,pitcher,pitch_type,season,n_pitches,velo,pfx_x,pfx_z,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage
0,434378,CH,2025,9,85.644444,-0.888889,0.731111,-5.925198,1.775221,6.388889,1525.222222,-1.893333,6.778889,-0.215556,1.161111,0.108434
1,434378,CU,2025,10,79.7,0.52,-0.816,-4.727961,1.769858,6.23,2720.9,-1.71,6.887,-0.033,2.398,0.120482
2,434378,FF,2025,43,94.1,-0.609535,1.480465,-4.203331,1.682875,6.202326,2459.069767,-1.586744,6.95814,0.008837,2.966744,0.518072
3,434378,SL,2025,10,88.18,0.387,0.458,-5.193331,2.307576,6.27,2451.4,-1.677,6.872,0.508,1.943,0.120482
4,434378,ST,2025,11,78.781818,1.045455,-0.449091,-5.015678,2.131764,6.154545,2642.545455,-2.001818,6.823636,0.020909,2.053636,0.132530


In [ ]:
# Merging with stats/descriptors:
final = pd.merge(agg, desc, on=['pitcher', 'pitch_type', 'season'])
final.head()

,pitcher,pitch_type,season,pitches,swings,whiffs,called_strikes,balls,whiff_pct,csw_pct,...,pfx_z,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage
0,434378,CH,2025,9,2,1,1,6,0.500000,0.222222,...,0.731111,-5.925198,1.775221,6.388889,1525.222222,-1.893333,6.778889,-0.215556,1.161111,0.108434
1,434378,CU,2025,10,3,1,4,3,0.333333,0.500000,...,-0.816,-4.727961,1.769858,6.23,2720.9,-1.71,6.887,-0.033,2.398,0.120482
2,434378,FF,2025,43,22,4,7,14,0.181818,0.255814,...,1.480465,-4.203331,1.682875,6.202326,2459.069767,-1.586744,6.95814,0.008837,2.966744,0.518072
3,434378,SL,2025,10,7,2,1,2,0.285714,0.300000,...,0.458,-5.193331,2.307576,6.27,2451.4,-1.677,6.872,0.508,1.943,0.120482
4,434378,ST,2025,11,4,0,3,4,0.000000,0.272727,...,-0.449091,-5.015678,2.131764,6.154545,2642.545455,-2.001818,6.823636,0.020909,2.053636,0.132530


In [ ]:
# final.to_parquet('initial_short_pitches_description_whiff_csw_stats.parquet')

In [7]:
final=pd.read_parquet(r"../../data/2023-03-30_2025-09-30_pitches_description_whiff_csw_stats.parquet")

In [8]:
# remove unneccessary columns
filtered_pitches = final.drop(columns=['swings', 'whiffs', 'called_strikes', 'balls', 'n_pitches'])

In [9]:
filtered_pitches.head()

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,p_throws,velo,pfx_x,pfx_z,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage
0,425794,CH,2023,89,0.100000,0.134831,R,81.431461,-1.168652,0.729663,-4.546824,0.9675,6.511236,1655.179775,-1.335618,6.14236,-0.424157,1.84809,0.049860
1,425794,CS,2023,3,0.000000,0.000000,R,65.733333,1.213333,-1.253333,-3.611843,0.728728,6.0,2448.666667,-1.52,6.41,-0.826667,2.97,0.001681
2,425794,CU,2023,546,0.149378,0.252747,R,71.506044,1.380531,-1.129249,-4.591665,1.387363,6.32334,2647.908918,-1.24359,6.284817,0.068315,1.933095,0.305882
3,425794,FC,2023,404,0.146597,0.220297,R,82.852228,0.464802,0.639381,-4.121794,1.857984,6.491688,2321.953368,-1.215272,6.287401,0.537302,2.394876,0.226331
4,425794,FF,2023,178,0.100000,0.297753,R,85.73427,-0.155562,1.235337,-3.745816,0.994451,6.530588,2158.635294,-1.232697,6.247135,-0.295618,2.713034,0.099720


In [10]:
# convert pfx_x and pfx_z to HB and and IVB in inches
filtered_pitches['HB'] = filtered_pitches['pfx_x'] * 12
filtered_pitches['IVB'] = filtered_pitches['pfx_z'] * 12
filtered_pitches.drop(columns=['pfx_x', 'pfx_z'], inplace=True)

In [11]:
filtered_pitches.head()

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,p_throws,velo,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage,HB,IVB
0,425794,CH,2023,89,0.100000,0.134831,R,81.431461,-4.546824,0.9675,6.511236,1655.179775,-1.335618,6.14236,-0.424157,1.84809,0.049860,-14.02382,8.755955
1,425794,CS,2023,3,0.000000,0.000000,R,65.733333,-3.611843,0.728728,6.0,2448.666667,-1.52,6.41,-0.826667,2.97,0.001681,14.56,-15.04
2,425794,CU,2023,546,0.149378,0.252747,R,71.506044,-4.591665,1.387363,6.32334,2647.908918,-1.24359,6.284817,0.068315,1.933095,0.305882,16.566374,-13.550989
3,425794,FC,2023,404,0.146597,0.220297,R,82.852228,-4.121794,1.857984,6.491688,2321.953368,-1.215272,6.287401,0.537302,2.394876,0.226331,5.577624,7.672574
4,425794,FF,2023,178,0.100000,0.297753,R,85.73427,-3.745816,0.994451,6.530588,2158.635294,-1.232697,6.247135,-0.295618,2.713034,0.099720,-1.866742,14.824045


In [12]:
filtered_pitches.columns

Index(['pitcher', 'pitch_type', 'season', 'pitches', 'whiff_pct', 'csw_pct',
       'p_throws', 'velo', 'VAA', 'HAA', 'ext', 'spin', 'rel_x', 'rel_z',
       'plate_x', 'plate_z', 'usage', 'HB', 'IVB'],
      dtype='object')

In [13]:
# first filter on number of pitches - to remove super small sample
filtered_pitches = filtered_pitches[filtered_pitches['pitches'] >= 30]

In [14]:
# Completely flatten out data for each pitcher
import pandas as pd
import numpy as np

def flatten_arsenal(
    df: pd.DataFrame,
    key_cols=("pitcher", "season"),
    pitch_col="pitch_type",
    feature_cols=("usage","velo","HB","IVB","VAA","HAA","spin","ext","rel_x","rel_z"),
    add_has_cols=True,
    keep_self_prefix=True,
    pitch_type_universe=None,   # optional list like ["FF","SI","SL",...]
):
    """
    Returns a dataframe with one row per (pitcher, season, pitch_type) that includes:
      - self_* columns (optional)
      - flattened arsenal columns: {PT}_{feat} for every pitch type PT
      - has_{PT} indicator columns (optional)

    df must already be aggregated so each (pitcher, season, pitch_type) is unique.
    """

    df = df.copy()

    # Ensure uniqueness at grain
    dup = df.duplicated(list(key_cols) + [pitch_col]).any()
    if dup:
        raise ValueError("Input df has duplicate (pitcher, season, pitch_type) rows. Aggregate first.")

    # Decide which pitch types to include as columns
    if pitch_type_universe is None:
        pitch_type_universe = sorted(df[pitch_col].dropna().unique().tolist())
    else:
        pitch_type_universe = list(pitch_type_universe)

    # --- 1) Build wide arsenal table: index=(pitcher, season), columns=(pitch_type, feature) ---
    wide = (
        df[list(key_cols) + [pitch_col] + list(feature_cols)]
        .pivot_table(
            index=list(key_cols),
            columns=pitch_col,
            values=list(feature_cols),
            aggfunc="first"
        )
    )

    # wide columns are a MultiIndex: (feature, pitch_type) -> flatten to "{PT}_{feature}"
    wide.columns = [f"{pt}_{feat}" for feat, pt in wide.columns]

    # Ensure every pitch type appears for every feature (even if missing in data)
    # This keeps a stable schema for your app/models.
    for pt in pitch_type_universe:
        for feat in feature_cols:
            col = f"{pt}_{feat}"
            if col not in wide.columns:
                wide[col] = np.nan

    # Reorder columns in a consistent way: by pitch type then feature
    ordered_cols = [f"{pt}_{feat}" for pt in pitch_type_universe for feat in feature_cols]
    wide = wide[ordered_cols].sort_index()

    # --- 2) Optional has_{PT} indicators (1 if usage exists and > 0) ---
    if add_has_cols:
        has = pd.DataFrame(index=wide.index)
        for pt in pitch_type_universe:
            ucol = f"{pt}_usage" if "usage" in feature_cols else None
            if ucol and ucol in wide.columns:
                has[f"has_{pt}"] = (wide[ucol].fillna(0) > 0).astype(int)
            else:
                # fallback: if any feature for that pitch type exists
                any_feat = [f"{pt}_{feat}" for feat in feature_cols if f"{pt}_{feat}" in wide.columns]
                has[f"has_{pt}"] = wide[any_feat].notna().any(axis=1).astype(int)

        wide = pd.concat([wide, has], axis=1)

    # --- 3) Create the row-level dataset (one row per pitch) and merge wide arsenal features onto it ---
    # Keep targets/other columns (e.g., whiff_pct/csw_pct) in "rows"
    rows = df.copy()

    # Attach the wide arsenal features (same for every pitch row within pitcher-season)
    out = rows.merge(wide.reset_index(), on=list(key_cols), how="left")

    # --- 4) Optional: create self_* columns so the model can easily refer to the evaluated pitch ---
    if keep_self_prefix:
        for feat in feature_cols:
            if feat in out.columns:
                out[f"self_{feat}"] = out[feat]

    return out
flattened = flatten_arsenal(
    filtered_pitches,
    key_cols=("pitcher", "season"),
    pitch_col="pitch_type",
    feature_cols=("usage","velo","HB","IVB","VAA","HAA","spin","ext","rel_x","rel_z"),
    add_has_cols=True,
    keep_self_prefix=True,
    pitch_type_universe=None
)

In [15]:
flattened.head()

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,p_throws,velo,VAA,HAA,...,self_usage,self_velo,self_HB,self_IVB,self_VAA,self_HAA,self_spin,self_ext,self_rel_x,self_rel_z
0,425794,CH,2023,89,0.100000,0.134831,R,81.431461,-4.546824,0.9675,...,0.049860,81.431461,-14.02382,8.755955,-4.546824,0.9675,1655.179775,6.511236,-1.335618,6.14236
1,425794,CU,2023,546,0.149378,0.252747,R,71.506044,-4.591665,1.387363,...,0.305882,71.506044,16.566374,-13.550989,-4.591665,1.387363,2647.908918,6.32334,-1.24359,6.284817
2,425794,FC,2023,404,0.146597,0.220297,R,82.852228,-4.121794,1.857984,...,0.226331,82.852228,5.577624,7.672574,-4.121794,1.857984,2321.953368,6.491688,-1.215272,6.287401
3,425794,FF,2023,178,0.100000,0.297753,R,85.73427,-3.745816,0.994451,...,0.099720,85.73427,-1.866742,14.824045,-3.745816,0.994451,2158.635294,6.530588,-1.232697,6.247135
4,425794,SI,2023,560,0.071429,0.260714,R,86.875,-4.051163,1.253781,...,0.313725,86.875,-13.252929,11.164071,-4.051163,1.253781,2167.058052,6.551685,-1.284518,6.134482


In [16]:
flattened.to_parquet(r"../../data/2023-03-30_2025-09-30_pitches_description_whiff_csw_stats_flattened.parquet")

In [26]:
import numpy as np
import pandas as pd

FASTBALL_TYPES_DEFAULT = ("FF", "SI", "FC", "FT")  # you can tweak this


def add_arsenal_context_features(
    df: pd.DataFrame,
    fastball_types=FASTBALL_TYPES_DEFAULT,
    fb_min_usage: float = 0.0,
) -> pd.DataFrame:
    """
    Adds arsenal context features to a season-level pitch table.

    Expected input columns (your filtered_pitches):
      ['pitcher','pitch_type','season','usage','velo','HB','IVB','VAA','HAA','ext','spin','rel_x','rel_z', ...]
    Output:
      - fb_* columns (primary fastball descriptors per pitcher-season)
      - delta_* columns (pitch minus fb)
      - arsenal summaries: arsenal_size, second_pitch_usage, max_other_usage
      - redundancy: min_movdist_to_other, min_shape_dist_to_other

    Notes:
      - Primary fastball is chosen as the highest-usage pitch among fastball_types.
      - If no fastball exists, fb_* and deltas become NaN.
      - Uses per pitcher-season grouping.
    """

    req = {"pitcher", "pitch_type", "season", "usage", "velo", "HB", "IVB", "VAA", "HAA", "ext", "spin", "rel_x", "rel_z"}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    out = df.copy()
    key_cols = ["pitcher", "season"]

    # --- 1) Identify primary fastball per pitcher-season (highest usage among fastball types) ---
    fb = out[out["pitch_type"].isin(fastball_types)].copy()
    if fb_min_usage > 0:
        fb = fb[fb["usage"] >= fb_min_usage]

    # idx of max usage fb per group
    fb_idx = fb.groupby(key_cols)["usage"].idxmax()
    fb_primary = fb.loc[fb_idx, key_cols + ["pitch_type", "usage", "velo", "HB", "IVB", "VAA", "HAA", "ext", "spin", "rel_x", "rel_z"]].copy()

    fb_primary = fb_primary.rename(columns={
        "pitch_type": "fb_pitch_type",
        "usage": "fb_usage",
        "velo": "fb_velo",
        "HB": "fb_HB",
        "IVB": "fb_IVB",
        "VAA": "fb_VAA",
        "HAA": "fb_HAA",
        "ext": "fb_ext",
        "spin": "fb_spin",
        "rel_x": "fb_rel_x",
        "rel_z": "fb_rel_z",
    })

    out = out.merge(fb_primary, on=key_cols, how="left")

    # --- 2) Delta features vs primary fastball ---
    for base, fbcol in [
        ("velo", "fb_velo"),
        ("HB", "fb_HB"),
        ("IVB", "fb_IVB"),
        ("VAA", "fb_VAA"),
        ("HAA", "fb_HAA"),
        ("ext", "fb_ext"),
        ("spin", "fb_spin"),
        ("rel_x", "fb_rel_x"),
        ("rel_z", "fb_rel_z"),
    ]:
        out[f"delta_{base}_vs_fb"] = out[base] - out[fbcol]

    # movement-only distance in inches space
    out["movdist_vs_fb"] = np.sqrt(
        (out["HB"] - out["fb_HB"]) ** 2 + (out["IVB"] - out["fb_IVB"]) ** 2
    )

    # a slightly richer "shape distance" (scale velo so it isn't tiny vs inches; tweak scale if you want)
    velo_scale = 2.0  # mph -> "inch units" (rough heuristic; tune later)
    out["shape_dist_vs_fb"] = np.sqrt(
        (velo_scale * (out["velo"] - out["fb_velo"])) ** 2
        + (out["HB"] - out["fb_HB"]) ** 2
        + (out["IVB"] - out["fb_IVB"]) ** 2
    )

    # --- 3) Arsenal summary features (per pitcher-season) ---
    # arsenal size = number of pitch types with nontrivial usage
    out["arsenal_size"] = out.groupby(key_cols)["pitch_type"].transform("nunique")

    # second pitch usage (2nd highest usage in arsenal)
    def second_highest(s: pd.Series) -> float:
        vals = np.sort(s.to_numpy())
        return float(vals[-2]) if len(vals) >= 2 else np.nan

    out["second_pitch_usage"] = out.groupby(key_cols)["usage"].transform(second_highest)

    # max usage of any other pitch (excluding this pitch)
    out["max_other_usage"] = out.groupby(key_cols)["usage"].transform("max")  # group max
    # If current pitch is the max, we need the second max as "max_other_usage"
    # We'll compute group top2 and map.
    def top_two(s: pd.Series):
        v = np.sort(s.to_numpy())
        if len(v) == 1:
            return (v[-1], np.nan)
        return (v[-1], v[-2])

    top2 = out.groupby(key_cols)["usage"].apply(top_two).reset_index(name="top2")
    top2[["g_max_usage", "g_second_usage"]] = pd.DataFrame(top2["top2"].tolist(), index=top2.index)
    top2 = top2.drop(columns=["top2"])

    out = out.merge(top2, on=key_cols, how="left")

    # If this pitch is the group's max usage, max_other_usage = group second. Else = group max.
    out["max_other_usage"] = np.where(
        np.isclose(out["usage"], out["g_max_usage"]),
        out["g_second_usage"],
        out["g_max_usage"]
    )
    out = out.drop(columns=["g_max_usage", "g_second_usage"])

    # --- 4) Redundancy features: min distance to any OTHER pitch in the arsenal ---
    # We'll compute per pitcher-season by brute-force within each group (groups are small: usually <= 6 pitches).
    def add_min_distances(group: pd.DataFrame) -> pd.DataFrame:
        hb = group["HB"].to_numpy()
        ivb = group["IVB"].to_numpy()
        velo = group["velo"].to_numpy()

        n = len(group)
        if n <= 1:
            group["min_movdist_to_other"] = np.nan
            group["min_shape_dist_to_other"] = np.nan
            return group

        # Pairwise movement distances
        d_mov = np.sqrt((hb[:, None] - hb[None, :]) ** 2 + (ivb[:, None] - ivb[None, :]) ** 2)

        # Pairwise shape distances (with velo scaled)
        d_shape = np.sqrt((velo_scale * (velo[:, None] - velo[None, :])) ** 2 +
                          (hb[:, None] - hb[None, :]) ** 2 +
                          (ivb[:, None] - ivb[None, :]) ** 2)

        # ignore self-distances by setting diagonal to inf
        np.fill_diagonal(d_mov, np.inf)
        np.fill_diagonal(d_shape, np.inf)

        group["min_movdist_to_other"] = d_mov.min(axis=1)
        group["min_shape_dist_to_other"] = d_shape.min(axis=1)
        return group

    out = out.groupby(key_cols, group_keys=False).apply(add_min_distances)

    return out


arsenal_df = add_arsenal_context_features(filtered_pitches)
arsenal_df.head()


/var/folders/h6/_rz_hgwd06q2j4rsml2mr9yr0000gn/T/ipykernel_37285/3942478890.py:153: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = out.groupby(key_cols, group_keys=False).apply(add_min_distances)


,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,velo,VAA,HAA,ext,...,delta_spin_vs_fb,delta_rel_x_vs_fb,delta_rel_z_vs_fb,movdist_vs_fb,shape_dist_vs_fb,arsenal_size,second_pitch_usage,max_other_usage,min_movdist_to_other,min_shape_dist_to_other
0,434378,FF,2025,43,0.181818,0.255814,94.1,-4.203331,1.682875,6.202326,...,0.0,0.0,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN
1,450203,FF,2025,32,0.230769,0.187500,94.509375,-2.175372,2.147469,5.996875,...,0.0,0.0,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN
2,518876,FF,2025,34,0.100000,0.205882,91.579412,-3.014159,2.076334,5.923529,...,0.0,0.0,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN
3,592332,FF,2025,52,0.242424,0.326923,95.390385,-3.176003,2.330984,6.35,...,0.0,0.0,0.0,0.0,0.0,2,0.350515,0.350515,13.224025,21.102807
4,592332,FS,2025,34,0.111111,0.088235,87.167647,-4.349028,2.650168,6.508824,...,-397.726244,-0.129016,-0.142851,13.224025,21.102807,2,0.350515,0.536082,13.224025,21.102807


In [31]:
arsenal_df[arsenal_df['pitch_type']=='SL']

,pitcher,pitch_type,season,pitches,whiff_pct,csw_pct,velo,VAA,HAA,ext,...,delta_spin_vs_fb,delta_rel_x_vs_fb,delta_rel_z_vs_fb,movdist_vs_fb,shape_dist_vs_fb,arsenal_size,second_pitch_usage,max_other_usage,min_movdist_to_other,min_shape_dist_to_other
10,625643,SL,2025,31,0.285714,0.064516,84.2,-4.876859,2.597133,6.454839,...,248.728111,-0.201389,-0.089322,23.822072,32.915496,2,0.329787,0.521277,23.822072,32.915496
12,641793,SL,2025,31,0.222222,0.290323,86.5,-3.575841,1.135679,6.254839,...,<NA>,<NA>,<NA>,<NA>,<NA>,1,NaN,NaN,NaN,NaN
14,656288,SL,2025,45,0.250000,0.266667,87.375556,-4.31016,1.555768,5.862222,...,<NA>,<NA>,<NA>,<NA>,<NA>,1,NaN,NaN,NaN,NaN
15,656302,SL,2025,43,0.318182,0.232558,89.637209,-4.556503,1.62318,6.27907,...,<NA>,<NA>,<NA>,<NA>,<NA>,1,NaN,NaN,NaN,NaN
28,681857,SL,2025,30,0.307692,0.366667,84.883333,-4.852074,3.06202,5.763333,...,<NA>,<NA>,<NA>,<NA>,<NA>,1,NaN,NaN,NaN,NaN
